# Лабораторна робота номер 2
### ФБ-24 Довгаль Анна

1. Створити env в якому будуть встановлені всі необхідні бібліотеки та
налаштування для данної лабораторної роботи


```conda
conda create --name adenv python
conda activate adenv
conda install pandas
conda install urllib.request
```

2. Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура
має бути автоматизована, параметром процедури має бути індекс
(номер) області. При зберіганні файлу до його імені потрібно додати
дату та час завантаження;

In [ ]:
import urllib
import datetime
import os

def get_csv(i):
    if not os.path.exists('csv'):
        os.makedirs('csv')
        
    url='https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={}&year1=1981&year2=2024&type=Mean'.format(i)
    wp = urllib.request.urlopen(url)
    text = wp.read()
    now = datetime.datetime.now()
    date_and_time = now.strftime("%d-%m-%Y_%H-%M-%S")
    out = open('.\\csv\\NOAA_ID_'+str(i)+'_'+date_and_time+'.csv','wb')
    out.write(text)
    out.close()

for i in range(1,28):
    get_csv(i)

3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу
із фреймами буде розказано у подальших лабораторних роботах).
Імена стовбців фрейму мають бути змістовними та легкими для
сприйняття (не повинно бути спеціалізованих символів, пробілів
тощо). Ця задача має бути реалізована у вигляді окремої процедури,
яка на вхід приймає шлях до директорії, в якій зберігаються файли;


In [ ]:
import pandas as pd
import os


def to_df(path):
    dfs=[]
    for file in os.listdir(path):
        if file.endswith('.csv'):
            headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI']
            df = pd.read_csv(os.path.join(path, file), header = 1, index_col=False, names = headers)
            parts = file.split("_")
            index = int(parts[2])
            df['Area'] = index

            dfs.append(df)

    df = pd.concat(dfs).drop_duplicates().reset_index(drop=True)

    df = df.drop(df.loc[df['VHI'] == -1].index)
    df.replace({"<tt><pre>1982": "1982"}, inplace=True)
    df.drop(df[df['Year'] == '</pre></tt>'].index, inplace=True)
    df['Year'] = pd.to_numeric(df['Year'])
    return df
    


df = to_df('.\\csv\\')
df.head()


In [ ]:
df.tail()

4. Реалізувати процедуру, яка змінить індекси областей, які використані
на порталі NOAA на дані:

In [ ]:
def convert_ids(df):
    correct_ids = {1 : 22, 2: 24, 3: 23, 4: 25, 5: 3, 6: 4, 7: 8, 8: 19, 9: 20, 10: 21, 11: 9, 12: 26, 13: 10, 14: 11, 15: 12, 16: 13, 17: 14, 18: 15, 19: 16, 20: 27, 21: 17, 22: 18, 23: 6, 24: 1, 25: 2, 26: 7, 27: 5}
    df['Area'].replace(correct_ids, inplace = True)
    return df

convert_ids(df)
df.head()

5. Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):
- Ряд VHI для області за вказаний рік, пошук екстремумів (min та
max);
- Ряд VHI за вказаний діапазон років для вказаних областей;
виявити роки, протягом яких екстремальні посухи торкнулися
більше вказаного відсотка областей по Україні (20% областей -
5 областей з 25);
- Аналогічно для помірних посух

In [ ]:
def get_extremum(df, area, year):
    min_v = df[(df["Area"] == area) & (df["Year"] == year)]['VHI'].min()
    max_v = df[(df["Area"] == area) & (df["Year"] == year)]['VHI'].max()
    return min_v, max_v
    

selected = df[(df["Area"] == 7) & (df["Year"] == 2005)]['VHI']

min, max = get_extremum(df, 21, 1982)
print('min: ', min, '\nmax: ', max)

In [ ]:
filtered = df[(df['Year'] >= 2000) & (df['Year'] <= 2003) & (df['Area'].isin([2, 6, 7]))]
print(filtered[['Year', 'Area', 'VHI']])

In [ ]:
def ext_droughts(df, pers):
    total = 27  
    years = df[df['VHI'] <= 15].groupby('Year')['Area'].nunique() #рахує кількість областей на кожен рік з посухою
    overpers = years[years > total * (pers / 100)]
    return overpers
    
persnt = 20
print(f"Роки з екстремальними посухами, які торкнулися більше {persnt}% областей по Україні:\n{ext_droughts(df, persnt)}")

In [ ]:
def mild_droughts(df, pers):
    total = 27  
    years = df[(df['VHI'] <= 35) & (df['VHI'] > 15)].groupby('Year')['Area'].nunique() 
    overpers = years[years > total * (pers / 100)]
    return overpers
    
pers = 90
print(f"Роки з помірними посухами, які торкнулися більше {pers}% областей по Україні:\n{mild_droughts(df, pers)}")

In [ ]:
df.to_csv("lab2.csv")